#### Задача 1: сравнение предложений
Дан набор предложений, скопированных с Википедии. Каждое из них имеет "кошачью тему" в одном из трех смыслов:

кошки (животные)
UNIX-утилита cat для вывода содержимого файлов
версии операционной системы OS X, названные в честь семейства кошачьих
Ваша задача — найти два предложения, которые ближе всего по смыслу к расположенному в самой первой строке. В качестве меры близости по смыслу мы будем использовать косинусное расстояние.

sentences.txt
Выполните следующие шаги:

Скачайте файл с предложениями (sentences.txt).
Каждая строка в файле соответствует одному предложению. Считайте их, приведите каждую к нижнему регистру с помощью строковой функции lower().
Произведите токенизацию, то есть разбиение текстов на слова. Для этого можно воспользоваться регулярным выражением, которое считает разделителем любой символ, не являющийся буквой: re.split('[^a-z]', t). Не забудьте удалить пустые слова после разделения.
Составьте список всех слов, встречающихся в предложениях. Сопоставьте каждому слову индекс от нуля до (d - 1), где d — число различных слов в предложениях. Для этого удобно воспользоваться структурой dict.
Создайте матрицу размера n * d, где n — число предложений. Заполните ее: элемент с индексом (i, j) в этой матрице должен быть равен количеству вхождений j-го слова в i-е предложение. У вас должна получиться матрица размера 22 * 254.
Найдите косинусное расстояние от предложения в самой первой строке (In comparison to dogs, cats have not undergone...) до всех остальных с помощью функции scipy.spatial.distance.cosine. Какие номера у двух предложений, ближайших к нему по этому расстоянию (строки нумеруются с нуля)? Эти два числа и будут ответами на задание. Само предложение (In comparison to dogs, cats have not undergone... ) имеет индекс 0.
Запишите полученные числа в файл, разделив пробелом. Обратите внимание, что файл должен состоять из одной строки, в конце которой не должно быть переноса. Пример файла с решением вы можете найти в конце задания (submission-1.txt).
Совпадают ли ближайшие два предложения по тематике с первым? Совпадают ли тематики у следующих по близости предложений?
Разумеется, использованный вами метод крайне простой. Например, он не учитывает формы слов (так, cat и cats он считает разными словами, хотя по сути они означают одно и то же), не удаляет из текстов артикли и прочие ненужные слова. Позже мы будем подробно изучать анализ текстов, где выясним, как достичь высокого качества в задаче поиска похожих предложений.

In [161]:
from numpy import zeros, dot, savetxt
from numpy.linalg import norm
import re

In [162]:
with open("sentences.txt") as f:
        # Определяем число предложений
        lines = sum(1 for _ in f)

In [163]:
with open("sentences.txt") as f:
    words = {}
    lcount = 0 # Индекс предложения
    wcount = 0 # Индекс уникального слова

    for line in f:
        tokens = re.split("[^a-z]+", line.lower()) # Приводим текст к нижнему регистру и разбиваем его на слова 
        tokens.pop() # Имеет только одно "пустое" слово, оно находится в конце списка - удаляем его

        for token in tokens:

            # Заполняем словарь words, значения - тоже словари
            # index - индекс слова, уникальное значение для каждогослова
            # occurrences - список, длина которого равна числу предложений. Значения списка - значения повторений числа в конкретном предложении
            if token not in words:
                words[token] = {
                    "index": wcount,
                    "occurrences": [0] * lines
                }
                wcount += 1

            # Если встречаем то же слово в том же прдложении в tokens - игнорируем
            elif words[token]["occurrences"][lcount] != 0:
                continue
                
            # Регистрируем сколько раз слово встречалось в предложении в tokens    
            words[token]["occurrences"][lcount] = tokens.count(token)    
        
        lcount += 1
#print(words)

In [164]:
arr = zeros((lines, len(words))) # Создаем матрицу 22х254 из нулей
        
# Для каждого слова словаря words берем число его повторений в кадом предложении и добавляем это значение в матрицу
for word in words:
    i, j = 0, words[word]["index"]
    for occ in words[word]["occurrences"]:
        arr[i, j] = occ
        i += 1

In [165]:
# Функция расчета косинусного расстояния
def cosine_distance(u, v):
    return 1.0 - (dot(u, v) / (norm(u) * norm(v)))

In [166]:
dist = [] 
u = arr[0,] 
for i in range(1, lines):
    v = arr[i,]
    dist.append({"index": i, "distance": cosine_distance(u, v)})    
        
dist.sort(key=lambda x: x["distance"])
print("Для предложения в 1-й строке является ближайшим - #%d с косинусным расстоянием %.2f.\n"\
"Для предложения в 1-й строке является вторым ближайшим - #%d с косинусным расстоянием %.2f." % (
    dist[0]["index"],
    dist[0]["distance"],
    dist[1]["index"],
    dist[1]["distance"]
))

Для предложения в 1-й строке является ближайшим - #6 с косинусным расстоянием 0.73.
Для предложения в 1-й строке является вторым ближайшим - #4 с косинусным расстоянием 0.78.
